In [3]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

np.random.seed(0)

X, Y = spiral_data(samples=100, classes=3)

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.02 * np.random.rand(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, input):
        self.output = np.dot(input, self.weights) + self.biases
        
class ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        # minus all the values in a single row with the max to stabilize everything
        # the max becomes 0 and others below 0
        # it doesnt effect the result because the softmax output is relative to each other
        # so [7,10,11] is the same as [-4, -1, 0]. scale wise
        norm_values = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = norm_values

class Loss:
    def calculate(self, y_pred, y_true):
        losses = self.forward(y_pred, y_true)
        total_loss = np.mean(losses) # all types of loss calculation ends up 'mean' ing all losses from each sample
        return total_loss
    
    
    
class CategoricalCrossEntropy(Loss):
    def forward(self, output, y):
        length = len(output)
        clipped = np.clip(output, 1e-7, 1-1e-7)
        if len(y.shape) == 1:
            new_values = clipped[range(length), y]
        elif len(y.shape) == 2:
            new_values = np.sum(clipped * y, axis=1)
        log_values = -np.log(new_values)
        return log_values

class Accuracy:
    def forward(self, y_pred, y_true):
        predictions = np.argmax(y_pred, axis=1)
        new_y = y_true
        if len(y_true.shape) == 2:
            new_y = np.argmax(y_true, axis=1)
        
        self.acc = np.mean(predictions==new_y)

dense1 = Layer_Dense(2,3)
dense1.forward(X)
activation1 = ReLU()
activation1.forward(dense1.output)

dense2 = Layer_Dense(3,3)
dense2.forward(activation1.output)
activation2 = Softmax()
activation2.forward(dense2.output)

lossFunc = CategoricalCrossEntropy()
loss = lossFunc.calculate(activation2.output, Y)
print(loss)
accuracy = Accuracy()
accuracy.forward(activation2.output, Y)
print(accuracy.acc)

1.0986087
0.36


# Formula practices

In [21]:
import numpy as np

array = np.random.rand(100)

output = np.array([1/i**2 for i in array])


print(array[:5])
print(output[:5])


[0.03294545 0.3101955  0.51543309 0.41595331 0.23125495]
[921.31714022  10.39271502   3.76405026   5.77977352  18.69897985]


In [41]:
import numpy as np

a = np.array([3,4])
b = np.array([0,0])

euclidean = np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)
print(euclidean)

5.0
